# Simulación Monte Carlo para Blackjack

## Banco de Guatemala
## Programa de Estudios Superiores -PES-, 2021-2022

*Programación I*

*Profesor: Rodrigo Chang*

*Fecha: 21 de Enero de 2022*

    Luis González

    Javier Monroy

### Paquetes necesarios

In [2]:
import numpy as np
import pandas as pd

### Funciones auxiliares

In [3]:
def arrays_to_df(pfh:np.ndarray, res:np.ndarray, dfh:np.ndarray):
    s_pfh=pd.Series(pfh)
    s_res=pd.Series(res)
    s_dfh=pd.Series(dfh)
    basic_strategy_stand = pd.DataFrame({'Hard_Total':s_pfh, 'Result':s_res, 'Dealer_Upcard':s_dfh})
    return basic_strategy_stand

### El juego de Blackjack

In [4]:
def blackjack_game(decks:int, players=1):    
    profit_dealer=0
    profit_player=0
    shoe=np.full(decks*4,11)
    two=np.full(decks*4,2)
    three=np.full(decks*4,3)
    four=np.full(decks*4,4)
    five=np.full(decks*4,5)
    six=np.full(decks*4,6)
    seven=np.full(decks*4,7)
    eight=np.full(decks*4,8)
    nine=np.full(decks*4,9)
    ten=np.full(decks*4,10)
    j=np.full(decks*4,10)
    q=np.full(decks*4,10)
    k=np.full(decks*4,10)
    total_faces=(two, three, four, five, six, seven, eight, nine, ten, j, q, k)
    for i in total_faces:
        shoe = np.hstack((shoe,i))
    np.random.shuffle(shoe)
    player_hand=np.full(1,0)
    dealer_hand=np.full(1,0)
    player_first_cards = np.full(1,0)
    dealer_first_cards = np.full(1,0)
    hand_result=np.full(1,0)
    while (shoe.size>5):        
        card = int(shoe[0])
        if (player_hand.size<=2):
            player_hand=np.append(player_hand, shoe[0])
        shoe = np.delete(shoe,0)                
        card = int(shoe[0])        
        if (dealer_hand.size<=2):
            dealer_hand=np.append(dealer_hand, shoe[0])        
        shoe = np.delete(shoe,0)         
        if (player_hand.size>=3) and (dealer_hand.size>=3):            
            while (np.sum(dealer_hand)<17):
                card = int(shoe[0])                
                dealer_hand=np.append(dealer_hand, shoe[0])  
                shoe = np.delete(shoe,0)                                        
            if (np.sum(dealer_hand)==21) and (np.sum(player_hand)!=21):
                profit_dealer+=1
                profit_player-=1
                player_first_cards=np.append(player_first_cards,np.sum(player_hand))
                dealer_first_cards=np.append(dealer_first_cards, dealer_hand[1])                
                hand_result=np.append(hand_result, -1)                
                player_hand=np.full(1,0)
                dealer_hand=np.full(1,0)                                
            elif (np.sum(dealer_hand) > np.sum(player_hand)) and (np.sum(dealer_hand)<21):                
                profit_dealer+=1
                profit_player-=1
                player_first_cards=np.append(player_first_cards,np.sum(player_hand))
                dealer_first_cards=np.append(dealer_first_cards, dealer_hand[1])
                hand_result=np.append(hand_result, -1)
                player_hand=np.full(1,0)
                dealer_hand=np.full(1,0)                                
            elif (np.sum(dealer_hand) == np.sum(player_hand)): 
                player_first_cards=np.append(player_first_cards,np.sum(player_hand))
                dealer_first_cards=np.append(dealer_first_cards, dealer_hand[1])
                hand_result=np.append(hand_result, 0)
                player_hand=np.full(1,0)
                dealer_hand=np.full(1,0)                                
            elif (np.sum(player_hand)==21) and (np.sum(dealer_hand)!=21):                
                profit_dealer-=1.5
                profit_player+=1.5
                player_first_cards=np.append(player_first_cards,np.sum(player_hand))
                dealer_first_cards=np.append(dealer_first_cards, dealer_hand[1])
                hand_result=np.append(hand_result, 1.5)
                player_hand=np.full(1,0)
                dealer_hand=np.full(1,0)                                
            elif (np.sum(dealer_hand) < np.sum(player_hand)) and (np.sum(player_hand)<21):                
                profit_dealer-=1
                profit_player+=1
                player_first_cards=np.append(player_first_cards,np.sum(player_hand))
                dealer_first_cards=np.append(dealer_first_cards, dealer_hand[1])
                hand_result=np.append(hand_result, 1)
                player_hand=np.full(1,0)
                dealer_hand=np.full(1,0)                                
            elif (np.sum(dealer_hand) > 21) and (np.sum(player_hand)<21):                
                profit_dealer-=1
                profit_player+=1
                player_first_cards=np.append(player_first_cards,np.sum(player_hand))
                dealer_first_cards=np.append(dealer_first_cards, dealer_hand[1])
                hand_result=np.append(hand_result, 1)
                player_hand=np.full(1,0)
                dealer_hand=np.full(1,0)                              
    else:
        pass         
    return (arrays_to_df(player_first_cards, hand_result, dealer_first_cards))

### El experimento Monte Carlo

In [5]:
def monte_carlo_blackjack(n:int, N:int):
    Basic_Strategy = pd.DataFrame()
    for _ in range(N):
        BJiteration = blackjack_game(n)
        Basic_Strategy=pd.concat([Basic_Strategy,BJiteration], ignore_index=True)
    Basic_Strategy = Basic_Strategy.iloc[1:,:]
    return Basic_Strategy

In [8]:
BJT = monte_carlo_blackjack(6,500)
BJT

,Hard_Total,Result,Dealer_Upcard
1,10,-1.0,10
2,19,0.0,10
3,15,-1.0,6
4,21,1.5,11
5,6,1.0,7
...,...,...,...
28070,13,-1.0,10
28071,14,-1.0,10
28072,20,1.0,4
28073,9,-1.0,10


### Nuestra Basic Strategy

In [10]:
Gamblers_Strategy = BJT.pivot_table(index="Hard_Total",columns="Dealer_Upcard").iloc[1:18,1:]
Gamblers_Strategy

Result                                                    \
Dealer_Upcard        2         3         4         5         6         7    
Hard_Total                                                                  
4             -0.111111  0.000000  0.400000  0.230769 -0.222222 -0.428571   
5             -0.448276 -0.034483 -0.266667 -0.032258  0.090909 -0.310345   
6              0.121951 -0.170732 -0.437500 -0.103448  0.055556 -0.150000   
7             -0.365854 -0.033333  0.137931  0.000000 -0.137255 -0.481481   
8             -0.083333 -0.346154 -0.266667  0.060606 -0.076923 -0.321429   
9             -0.189189 -0.142857 -0.205882  0.152542  0.064935 -0.480519   
10            -0.406593 -0.178082 -0.010753 -0.061224 -0.200000 -0.236842   
11            -0.085106 -0.177778 -0.154639 -0.050505 -0.123810 -0.511111   
12            -0.187500 -0.177778 -0.149171 -0.095238 -0.060000 -0.487685   
13            -0.142857 -0.131313 -0.085427 -0.077626 -0.148718 -0.452915   
14            -0.175879 -0.162011 -0.097826  0.070064  0.016949 -0.371134   
15            -0.071823 -0.142857 -0.103448 -0.087912 -0.034091 -0.406977   
16            -0.314286 -0.220513 -0.087719  0.000000 -0.035294 -0.559322   
17            -0.117647 -0.050725  0.055172  0.062069  0.104046 -0.096154   
18             0.214286  0.218978  0.312977  0.253521  0.198582  0.414634   
19             0.577236  0.423358  0.489655  0.575221  0.630631  0.581967   
20             0.572139  0.688442  0.647541  0.658009  0.653226  0.779343   

                                                       
Dealer_Upcard        8         9         10        11  
Hard_Total                                             
4             -0.428571 -0.857143 -0.512195 -0.538462  
5             -0.750000 -0.407407 -0.607843 -0.485714  
6             -0.538462 -0.422222 -0.565217 -0.310345  
7             -0.640000 -0.566667 -0.528090 -0.649123  
8             -0.532468 -0.369863 -0.511811 -0.469388  
9             -0.365079 -0.558824 -0.554140 -0.428571  
10            -0.418605 -0.305263 -0.515670 -0.357143  
11            -0.592233 -0.495327 -0.570732 -0.570093  
12            -0.462687 -0.516129 -0.591837 -0.492063  
13            -0.462366 -0.443902 -0.521059 -0.517241  
14            -0.387283 -0.409836 -0.517241 -0.428571  
15            -0.446328 -0.437500 -0.513120 -0.555556  
16            -0.427027 -0.526882 -0.565868 -0.494624  
17            -0.371622 -0.368794 -0.458882 -0.308219  
18             0.046512 -0.205674 -0.250457 -0.141935  
19             0.636364  0.162162  0.002101 -0.048387  
20             0.741379  0.754310  0.409091  0.302632